# Add Atlas to Database
## Parameters
The next code block sets parameters that are used throughout the remainder of the notebook.

In [ ]:
# pylint: disable=invalid-name,missing-module-docstring

# atlas data in CSV file format
csv_atlas_file_name = None

# name to assign to atlas
atlas_name = None

# use 'positive' or 'negative'
polarity = None

# overrides any mz_tolerance value in the CSV file
mz_tolerance = None


# The rest of this block contains project independent parameters

# to use an older version of the metatlas source code, set this to a commit id,
# branch name, or tag. If None, then use the the "main" branch.
source_code_version_id = None

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = "INFO"

## Initialization

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
import logging  # noqa: E402
from pathlib import Path  # noqa: E402
from IPython.display import Markdown, display  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=ghcr.io/biorack/metatlas/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from metatlas.tools import notebook  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err
notebook.setup(log_level, source_code_version_id)
from metatlas.plots.dill2plots import make_atlas_from_spreadsheet  # noqa: E402
from metatlas.io.metatlas_get_data_helper_fun import make_atlas_df  # noqa: E402

## Atlas generation

In [ ]:
assert csv_atlas_file_name is not None
assert atlas_name is not None
atlas = make_atlas_from_spreadsheet(
    csv_atlas_file_name, atlas_name, filetype="csv", polarity=polarity, store=True, mz_tolerance=mz_tolerance
)
atlas_df = make_atlas_df(atlas)
display(Markdown(f"### Atlas unique_id: {atlas.unique_id}"))
display(atlas_df)